## Test per progetto AI

E' necessario caricare i file roads_small.json e nodes_small.json

In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import networkx as nx
import random

from datetime import datetime
from datetime import timedelta

In [4]:
nodes = json.load(open('new_nodes.json', 'r'))
roads = json.load(open('new_roads.json', 'r'))
print("Num nodes", len(nodes))
print("Num roads", len(roads))

Num nodes 2239
Num roads 10138


Create two maps as lookup tables for the nodes

In [5]:
#create a lookup table for point coordinates
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [6]:
print(node_coords[roads[11]['p1']]) #retrieve coordinates from node ID

(12.55, 45.59)


A questo punto bisogna generare le query degli utenti:

semplificazione: l'orario non influisce sui tempi di percorrenza -> dipendono solo dalla distanza

Consider each trip as starting or ending in one of the nodes: the function below serves to approximate the closest node on the map

Regular transfer (not related to pendolarism) is modeled as a gaussian random variable with respect to location ($\mu = 15km, \sigma = 10km$)  

In [7]:
# compute distance between two nodes

def compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2):
  x_coord_1 = x_coord_1 * 111.32
  x_coord_2 = x_coord_2 * 111.32
  y_coord_1 = 40075 * math.cos(x_coord_1)/360 * y_coord_1;
  y_coord_2 = 40075 * math.cos(x_coord_2)/360 * y_coord_2;
  dist = math.sqrt((y_coord_1 - y_coord_2)**2 + (x_coord_1 - x_coord_2)**2)
  return dist

In [8]:
def compute_distance(x1, y1, x2, y2) :
  R = 6371e3 #metres
  phi1 = x1 * math.pi/180 # φ, λ in radians
  phi2 = x2 * math.pi/180
  delta_phi = (x2-x1) * math.pi/180
  delta_lambda = (y2-y1) * math.pi/180

  a = math.sin(delta_phi/2) * math.sin(delta_phi/2) + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2) * math.sin(delta_lambda/2)
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));

  d = R * c #in metres

  return d

In [9]:
# use of networkx, creation of graph

G = nx.Graph()

G.add_nodes_from(range(len(nodes)))
for road in roads :
  n1 = int(road.get("p1"))
  n2 = int(road.get("p2"))
  x_coord_1 = float(nodes[n1].get('coordinates')[0])
  y_coord_1 = float(nodes[n1].get('coordinates')[1])
  x_coord_2 = float(nodes[n2].get('coordinates')[0])
  y_coord_2 = float(nodes[n2].get('coordinates')[1])
  dist = compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2)

  G.add_edge(n1, n2, weight=dist)

G.get_edge_data(0, 1)



{'weight': 1553.904995103261}

In [10]:
def generate_ratings(n_driver):
  mean, std = 3.5, 0.5
  x = np.random.normal(mean, std, size=(n_driver, 1))
  ratings = np.empty(shape = (n_driver, 1))
  for i in range(n_driver):
    y = math.ceil(x[i])
    if (y - x[i] > 0.5) :
      ratings[i] = y - 0.5
    else :
      ratings[i] = y
      
  return ratings


In [11]:
# pool of end points 
# define a random number of destinations (e.g. from 6 to 8)
num_dest = random.randrange(6, 8, 1)
pool_dest = []
for i in range(num_dest):
  rand = random.randrange(len(node_coords))
  id_dest = list(node_coords.keys())[rand]
  coord_dest = list(node_coords.values())[rand]
  pool_dest.append({'id':id_dest, 'coordinates':coord_dest})

print(pool_dest)

[{'id': 295, 'coordinates': (12.58, 45.47)}, {'id': 12, 'coordinates': (12.64, 45.54)}, {'id': 147, 'coordinates': (12.45, 45.45)}, {'id': 1768, 'coordinates': (11.91, 45.3)}, {'id': 955, 'coordinates': (12.16, 45.54)}, {'id': 1628, 'coordinates': (12.65, 45.65)}]


Minimum path lenght:
- Drivers: 40 km
- Riders: 10 km

In [12]:
def generate_trips(n, gen):
  # needs to define, for each driver/rider, one random starting point, and peak from the pool of destinations a random end points
  trips_start = []
  trips_dest = []
  for i in range(n):
    # start point
    if (gen == 'driver') :
      while True:
        # start point
        rand_s = random.randrange((len(nodes)))
        id_start = list(node_coords.keys())[rand_s]
        # end point
        rand_d = random.randrange(num_dest)
        dest = pool_dest[rand_d]
        dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
        if (dist > 40e3) :
          coord_start = list(node_coords.values())[rand_s]
          break
    else :
        while True:
          # start point
          rand_s = random.randrange((len(nodes)))
          id_start = list(node_coords.keys())[rand_s]
          # end point
          rand_d = random.randrange(num_dest)
          dest = pool_dest[rand_d]
          dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
          if (dist > 10e3) :
            coord_start = list(node_coords.values())[rand_s]
            break
     
    
    trips_start.append({'id':id_start, 'coordinates':coord_start})
    trips_dest.append(dest)

  return trips_start, trips_dest


In [13]:
def generate_schedules(n):
  
  #time probability mass: how many requests per 2-hour slot in percentage
  time_prob = {   
                  '00:00': 1e-3,
                  '2:00': 1e-3,
                  '4:00': 1e-3,
                  '6:00': 1e-3,
                  '6:30': 5e-3,
                  '7:00': 0.02,
                  '7:30': 0.1,
                  '8:00': 0.1,
                  '8:30': 0.1,
                  '10:00': 0.01,
                  '10:30': 0.01,
                  '12:00': 0.1,
                  '12:30': 0.1,
                  '13:00': 0.1,
                  '14:00': 0.01,
                  '16:00': 0.01,
                  '18:00': 0.1,
                  '18:30': 0.1,
                  '19:00': 0.1,
                  '19:30': 0.02,
                  '20:00': 5e-3,
                  '21:00': 5e-3,
                  '22:00': 1e-3,
  }

  

  times = []
  hours = list(time_prob.keys())
  prob = list(time_prob.values())
  for i in range(n):
    times.append((random.choices(hours, prob)[0]))

  return times

In [14]:
class Driver:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Driver ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

class Rider:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    date_format_str = '%H:%M'
    return "Rider ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

In [15]:
def generate_data():
  # 1- generate n_drivers and n_riders
  # range: start, end, stepsize
  n_drivers = random.randrange(40, 50, 1)
  n_riders = random.randrange(150, 250, 15)
  print("N_drivers: ", n_drivers)
  print("N_riders: ", n_riders)
  # 2 - generate start point and end point for drivers and riders
  trips_start_drivers, trips_dest_drivers = generate_trips(n_drivers, gen='driver')
  trips_start_riders, trips_dest_riders = generate_trips(n_riders, gen='rider')
  # 3 - generate schedules
  drivers_schedule = generate_schedules(n_drivers)
  riders_schedule = generate_schedules(n_riders)
  # 4- generate ratings for drivers and riders
  ratings_d = generate_ratings(n_drivers)
  ratings_r = generate_ratings(n_riders)
  # 5 - parse in a list of objects
  riders = []
  drivers = []
  for i in range(n_riders):
    riders.append( Rider(i, ratings_r[i][0], riders_schedule[i], trips_start_riders[i], trips_dest_riders[i]))

  for i in range(n_drivers):
    drivers.append( Driver(i, ratings_d[i][0], drivers_schedule[i], trips_start_drivers[i], trips_dest_drivers[i]))

  return drivers, riders



In [16]:
drivers, riders = generate_data()

for i in range(10) :
  print(drivers[i])

print("\n\n***********************************************************************\n\n")

for i in range(10) :
  print(riders[i])


N_drivers:  44
N_riders:  225
Driver ( id = 0, rating = 4.0, hour = 8:00, start = {'id': 530, 'coordinates': (12.34, 45.5)}, dest = {'id': 1768, 'coordinates': (11.91, 45.3)})
Driver ( id = 1, rating = 4.5, hour = 18:00, start = {'id': 366, 'coordinates': (12.31, 45.49)}, dest = {'id': 12, 'coordinates': (12.64, 45.54)})
Driver ( id = 2, rating = 4.0, hour = 8:30, start = {'id': 842, 'coordinates': (12.18, 45.22)}, dest = {'id': 1628, 'coordinates': (12.65, 45.65)})
Driver ( id = 3, rating = 5.0, hour = 8:00, start = {'id': 1547, 'coordinates': (12.0, 45.45)}, dest = {'id': 12, 'coordinates': (12.64, 45.54)})
Driver ( id = 4, rating = 3.5, hour = 12:00, start = {'id': 1130, 'coordinates': (11.98, 45.54)}, dest = {'id': 12, 'coordinates': (12.64, 45.54)})
Driver ( id = 5, rating = 4.0, hour = 7:30, start = {'id': 1069, 'coordinates': (12.1, 45.53)}, dest = {'id': 12, 'coordinates': (12.64, 45.54)})
Driver ( id = 6, rating = 4.0, hour = 8:30, start = {'id': 1421, 'coordinates': (11.77, 4

In [17]:
# use of Dijkstra for shortest path

drivers_path = []
riders_path = []

# 50 km/h
avg_vel = 50
date_format_str = '%H:%M'

for driver in drivers:
  spath = nx.dijkstra_path(G, driver.start_point.get('id'), driver.end_point.get('id'))
  dpath = [{'node' : spath[0], 'hour': driver.hour}]
  dist = 0


  for i in range(1, len(spath)):
    # retrieve distance and convert into km
    
    # distance in km
    dist += (G.get_edge_data(spath[i-1], spath[i]).get('weight')) / 1e3
    time = dist / avg_vel
    time_1 = datetime.strptime(driver.hour, date_format_str)
    time_2 = time_1 + timedelta(hours=time)
    str_time = time_2.strftime(date_format_str)

    dpath.append({'node': spath[i], 'hour':str_time})


  total_dist = nx.shortest_path_length(G, driver.start_point.get('id'), driver.end_point.get('id'), weight='weight')

  drivers_path.append({'id_driver': driver.id, 'shortest_path' : dpath, 'arrival_time': str_time})
  
  #print("Driver #" + str(driver.id) + ": start at " + driver.hour + " , end at " + dpath[len(dpath)-1].get('hour'))
  #print("    -> total dist km: ", total_dist/1e3)


for rider in riders:
  spath = nx.dijkstra_path(G, rider.start_point.get('id'), rider.end_point.get('id'))
  riders_path.append({'id_rider': rider.id, 'shortest_path' : spath})

print(drivers_path[0])
#print(riders_path)

{'id_driver': 0, 'shortest_path': [{'node': 530, 'hour': '8:00'}, {'node': 292, 'hour': '08:01'}, {'node': 365, 'hour': '08:02'}, {'node': 366, 'hour': '08:04'}, {'node': 234, 'hour': '08:05'}, {'node': 1004, 'hour': '08:08'}, {'node': 84, 'hour': '08:10'}, {'node': 27, 'hour': '08:12'}, {'node': 237, 'hour': '08:13'}, {'node': 121, 'hour': '08:14'}, {'node': 122, 'hour': '08:16'}, {'node': 331, 'hour': '08:17'}, {'node': 120, 'hour': '08:18'}, {'node': 536, 'hour': '08:20'}, {'node': 316, 'hour': '08:24'}, {'node': 647, 'hour': '08:25'}, {'node': 688, 'hour': '08:27'}, {'node': 131, 'hour': '08:28'}, {'node': 645, 'hour': '08:30'}, {'node': 706, 'hour': '08:32'}, {'node': 1541, 'hour': '08:33'}, {'node': 1542, 'hour': '08:35'}, {'node': 1986, 'hour': '08:36'}, {'node': 1898, 'hour': '08:38'}, {'node': 760, 'hour': '08:41'}, {'node': 757, 'hour': '08:43'}, {'node': 754, 'hour': '08:44'}, {'node': 744, 'hour': '08:45'}, {'node': 743, 'hour': '08:47'}, {'node': 685, 'hour': '08:48'}, {'n

In [18]:
def min_distance (shortest_path_driver, rider) :
  min_distance = float('inf')
  xr = rider.start_point.get('coordinates')[0]
  yr = rider.start_point.get('coordinates')[1]
  for node in shortest_path_driver :
    id = node.get('node')
    x1 = nodes[id].get('coordinates')[0]
    x2 = nodes[id].get('coordinates')[1]
    dist = compute_distance(x1, x2, xr, yr)
    if dist < min_distance :
      min_distance = dist

  return min_distance


In [19]:
# find matching: since we assumed drivers and riders having same dest
# first: filter the total of drivers and riders

drivers_match = [{} for _ in range(len(drivers))]

# couple drivers and riders with same dest and similar hours

for driver in drivers :
  drivers_match[driver.id] = {'id_driver' : driver.id, 'list_of_riders' : []}
  for rider in riders :
    dhour_start = datetime.strptime(driver.hour, date_format_str)
    dhour_end = datetime.strptime(drivers_path[driver.id].get('arrival_time'), date_format_str)
    rhour = datetime.strptime(rider.hour, date_format_str)

    #print("min distance: " , min_distance(drivers_path[driver.id].get('shortest_path'), rider))

    # filter per
    # 1- same dest
    # 2- simil hour
    # 3- minimum distance from rider and shortest path <= 30 km
    if driver.end_point.get('id') == rider.end_point.get('id') and ((dhour_start - timedelta(minutes=30)) <= rhour <= (dhour_end + timedelta(minutes=30)))  :
      drivers_match[driver.id].get('list_of_riders').append(rider.id)
  



In [39]:
def preference_list_drivers(drivers_match) :

  preferences = [{} for _ in range(len(drivers))]

  for driver in drivers :
    pref_dict = {}
    for r in drivers_match[driver.id].get('list_of_riders') :
      rider = riders[r]     
      min_dist = min_distance(drivers_path[driver.id].get('shortest_path'), rider)
      if min_dist <= 35e3 :
        # 0.4 * rating + 0.6 * distance in kms
        measure =  0.4*rider.rating + 0.6*(min_dist/1e3)
        pref_dict[rider.id] = measure

    # sort list
    preferences[driver.id] = {'id_driver' : driver.id, 'list_of_riders' : list(dict(sorted(pref_dict.items(), key=lambda item: item[1])))}

    
    
  return preferences



In [81]:
def preference_list_riders(list_of_riders) :

  preferences = [{} for _ in range(len(riders))]

  pref = [ {} for _ in range(len(riders))]

  # build the lists

  for driver in drivers:
    for r in drivers_match[driver.id].get('list_of_riders') :
      rider = riders[r]
      pref[rider.id][driver.id] = driver.rating
      

  #sort the lists

  for i in range(len(riders)):
    preferences[i] = {'id_rider' : i, 'list_of_drivers' : list(dict(sorted(pref[i].items(), key=lambda item: item[1])))} 
  
  return preferences

# Start of main algorithm

In [82]:
# 1- get path from Steiner tree (first option: compute shortest path)

# 2 - compute preference list of drivers

#print(drivers_match[0].get('list_of_riders'))

print(preference_list_drivers(drivers_match))

# 3 - compute preference list of riders

print(preference_list_riders(drivers_match))

[{'id_driver': 0, 'list_of_riders': [68, 169, 78, 218, 156]}, {'id_driver': 1, 'list_of_riders': [12, 208, 99, 117, 32]}, {'id_driver': 2, 'list_of_riders': [143, 73, 201, 63, 1, 122, 189, 139, 190, 16, 111]}, {'id_driver': 3, 'list_of_riders': [100, 112, 48, 217, 20, 76, 24, 154, 204, 106]}, {'id_driver': 4, 'list_of_riders': [43, 131, 93, 6, 181, 23, 140, 216, 121]}, {'id_driver': 5, 'list_of_riders': [48, 100, 217, 112, 167, 76, 20, 24, 204, 106, 154]}, {'id_driver': 6, 'list_of_riders': [75, 118, 158, 110]}, {'id_driver': 7, 'list_of_riders': []}, {'id_driver': 8, 'list_of_riders': [126, 150, 2, 172]}, {'id_driver': 9, 'list_of_riders': [131, 93, 6, 43, 23]}, {'id_driver': 10, 'list_of_riders': [75, 220, 158, 53, 118, 110]}, {'id_driver': 11, 'list_of_riders': [75, 220, 110, 118, 53, 158]}, {'id_driver': 12, 'list_of_riders': [75, 220, 118, 110, 53, 158]}, {'id_driver': 13, 'list_of_riders': [24, 217, 112, 106, 167, 100, 48, 20, 76, 107, 154]}, {'id_driver': 14, 'list_of_riders': [

In [ ]:
# create file input.dat (obsolete (?))
f = open("input.dat", "w")
content = "DIMENSION : " + str(len(nodes)) + "\n"
content += "DRIVERS : " + str(len(drivers)) + "\n"
content += "RIDERS : " + str(len(riders)) + "\n"
content += "NODE_COORD_SECTION\n"
for node in nodes :
  content += str(node.get("id")) + " " + str(node.get("coordinates")[0]) + " " + str(node.get("coordinates")[1]) + "\n"
content += "ROADS_SECTION\n"
for road in roads :
  content += str(road.get("p1")) + " " + str(road.get("p2")) + "\n"
content += "DRIVERS_STARTING_COORD\n"
for driver in drivers:
  content += str(driver.start_point.get("id")) + " " + str(driver.start_point.get("coordinates")[0]) + " " + str(driver.start_point.get("coordinates")[1]) + "\n"
content += "DRIVERS_DESTINATION_COORD\n"
for driver in drivers:
  content += str(driver.end_point.get("id")) + " " + str(driver.end_point.get("coordinates")[0]) + " " + str(driver.end_point.get("coordinates")[1]) + "\n"
content += "RIDERS_STARTING_COORD\n"
for rider in riders:
  content += str(rider.start_point.get("id")) + " " + str(rider.start_point.get("coordinates")[0]) + " " + str(rider.start_point.get("coordinates")[1]) + "\n"
content += "RIDERS_DESTINATION_COORD\n"
for rider in riders:
  content += str(rider.end_point.get("id")) + " " + str(rider.end_point.get("coordinates")[0]) + " " + str(rider.end_point.get("coordinates")[1]) + "\n"
content += "EOF"
f.write(content)
f.close()